In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import operator
import pickle 
import json

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import validation_curve
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, accuracy_score, r2_score
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import train_test_split
from ipywidgets import interact, fixed
#from xgboost import XGBRegressor, XGBClassifier
#from pdpbox import pdp
#from pdpbox.pdp import pdp_isolate, pdp_plot, pdp_interact, pdp_interact_plot
import eli5
from eli5.sklearn import PermutationImportance
import category_encoders as ce
#import shap

In [2]:
from KMeansCluster import centroids, kmeans, labels, new_series#, visualizer

danceability  energy  loudness  speechiness  acousticness  \
0         0.511   0.472    -9.277       0.0910         0.534   
1         0.742   0.546    -7.694       0.0315         0.172   
2         0.609   0.629    -5.024       0.0264         0.425   
3         0.730   0.770    -5.114       0.0298         0.138   
4         0.706   0.558    -7.237       0.0593         0.146   

   instrumentalness  liveness  valence    tempo  duration_ms  clusters  
0             0.614    0.1100   0.0399  127.978       258969         4  
1             0.380    0.1150   0.8680  104.996       260520         4  
2             0.000    0.0978   0.3250   99.955       241467         4  
3             0.000    0.0473   0.5070  104.948       228093         0  
4             0.000    0.1180   0.2270  110.443       223253         0  
          danceability    energy  loudness  speechiness  acousticness  \
clusters                                                                
0             0.598537  0.686120 -

TypeError: 'DataFrame' object is not callable

In [ ]:
track_master_df = pd.read_csv('track_master_df.csv')
track_master_df.shape

In [ ]:
track_master_df.head()

test_tracks = pd.read_csv('test_case.csv')

In [17]:
drop_cols = ['track']

X_train = track_master_df.drop(columns = drop_cols)

X_test = test_tracks.drop(columns = drop_cols)

In [18]:
knn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(X_train)

In [19]:
distances, indices = knn.kneighbors(X_test)

In [20]:
# Its important to use binary mode 
knn_pickle = open('knnpickled', 'wb') 

# source, destination 
pickle.dump(knn, knn_pickle)                      

In [21]:
# load the model
loaded_model = pickle.load(open('knnpickled', 'rb'))

distances, indices = loaded_model.kneighbors(X_test) 

In [22]:
indices

array([[5693, 8126, 7977, 5042, 6045],
       [8705, 3553, 2901, 6352, 6515],
       [8897, 4800, 4039, 8046, 8544],
       [4902, 4064, 1805, 3584, 9416],
       [ 282, 9078, 3728, 6810, 1877],
       [1541, 6254, 9505, 7190, 1536],
       [1021, 2645, 3074, 3678, 2393],
       [1752, 7300, 6257, 5395, 5807],
       [4811, 4285, 1940, 6662, 4304],
       [1808, 4188, 1044, 6815, 6972],
       [3293, 4055, 7476,  875, 4383],
       [5472, 8062, 8429, 4018, 6250],
       [9250, 5585,  413, 3967, 9476]])

In [23]:
distances

array([[39.5792277 , 45.51040963, 46.61482086, 50.45865301, 52.36851431],
       [ 0.        , 22.55081201, 43.19630838, 45.32310325, 45.65762443],
       [12.57199594, 22.1328905 , 43.00935584, 43.32750337, 50.41571789],
       [19.36216389, 25.34454699, 35.55630885, 53.58995679, 63.32647685],
       [ 0.        , 20.82312796, 30.40603846, 32.37235047, 34.89708647],
       [ 0.        , 15.93425035, 40.36314622, 43.05843212, 48.54124602],
       [ 0.        , 16.61446361, 31.37678678, 60.79655261, 67.57502377],
       [ 0.        , 64.3038712 , 64.66443145, 68.07058708, 75.62057531],
       [15.09146261, 26.1371749 , 32.46923998, 32.98035392, 35.49199129],
       [ 0.        ,  6.49446415, 13.30877887, 39.22663807, 40.21411076],
       [40.12189874, 40.23662806, 42.12032638, 45.53288   , 45.79620138],
       [30.03719626, 33.0794943 , 33.74515013, 34.45749801, 43.73032492],
       [ 3.08967913, 31.10158097, 34.58640959, 34.65045427, 46.6986893 ]])

In [24]:
recommend_indices = []
for ii, dists in enumerate(distances):
    for jj, val in enumerate(dists):
        if (val > 0) & (val < 50):
            recommend_indices.append((indices[ii][jj], int(round(val))))

recommend_indices = sorted(recommend_indices, key = operator.itemgetter(1))

In [25]:
ind, val = zip(*recommend_indices) 

In [26]:
recommendations = track_master_df.iloc[list(ind[:10])][['artist','album','track']]

recommendations

,artist,album,track
9250,Jaira Burns,Ugly,Ugly
4188,The Shadowboxers,Build The Beat,Build The Beat
8897,Halsey,hopeless fountain kingdom (Deluxe),Eyes Closed
1044,Unwritten Law,Funk Metal Favorites,Seein' Red
4811,Felipe Muñiz,Deje de Amar,Deje de Amar
6254,Common,Black America Again,Unfamiliar
2645,The Go-Go's,Return To The Valley Of The Go-Go's,This Town
4902,Fools Garden,Dish Of The Day,Lemon Tree
9078,The Detroit Emeralds,Greatest Hits,Baby Let Me Take You (In My Arms)
4800,Mackenzie Thoms,Shout,Shout


In [27]:
rec_json = recommendations.to_json(orient = 'table', index = False)

In [28]:
with open('recommendations.txt', 'w') as outfile:
    json.dump(rec_json, outfile)

In [86]:
# Train, test split
train, test = train_test_split(track_master_df, train_size=0.80, test_size=0.20, 
                              stratify=kmeans['mode'], 
                              random_state=42)

In [87]:
# Train, val split
train, val = train_test_split(new_series, train_size=0.80, test_size=0.20, 
                              #stratify=track_master_df['mode'], 
                              random_state=42)

In [94]:
# Target
target = 'mode'

# Get a dataframe with all train columns except the target
train_features = train.drop(columns=[target])

# Get a list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# Get a series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# Get a list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

AttributeError: 'Series' object has no attribute 'select_dtypes'

In [92]:
# Arrange data into X features matrix and y target vector 
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

KeyError: "None of [Index(['danceability', 'energy', 'key', 'loudness', 'speechiness',\n       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',\n       'duration_ms', 'time_signature'],\n      dtype='object')] are in the [index]"

In [90]:
# Random Forest with Ordinal Encoder
import category_encoders as ce

random_forest = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
)

random_forest.fit(X_train, y_train)
print('Validation Accuracy:', random_forest.score(X_val, y_val), 
      '\nTest Accuracy:', random_forest.score(X_test, y_test))

Validation Accuracy: 0.6854922279792746 
Test Accuracy: 0.6854922279792746


In [44]:
# Using eli5 to find permutation importances

import eli5
from eli5.sklearn import PermutationImportance

# transforming data for permuter
transformers = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median')
)

X_train_transformed = transformers.fit_transform(X_train)
X_val_transformed = transformers.fit_transform(X_val)

# setting model
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
model.fit(X_train_transformed, y_train)

# setting permuter to permutation importance
permuter = PermutationImportance(
    model, 
    scoring='accuracy',
    n_iter=3,
    random_state=42
)

# fitting the data
permuter.fit(X_val_transformed, y_val)
feature_names = X_val.columns.tolist()

eli5.show_weights(
    permuter,
    top=None,
    feature_names = feature_names
)

Weight,Feature
0.0408 ± 0.0079,key
0.0247 ± 0.0110,speechiness
0.0100 ± 0.0035,danceability
0.0098 ± 0.0059,acousticness
0.0095 ± 0.0034,liveness
0.0093 ± 0.0081,energy
0.0040 ± 0.0013,duration_ms
0.0040 ± 0.0056,tempo
0.0026 ± 0.0053,loudness
0.0017 ± 0.0010,instrumentalness
